# GPT-5
In this notebook, I will cover a quick look at some new capabilities and new features of GPT-5.

This includes:
+ New parameters - verbosity, reasoning effort, etc.
+ Tools
+ Simple online games
+ Constraint satisfaction - summer scheduling, dog problem.  
+ Image and text input modalities
    + Navigation
    + Counting
+ Model router


In [ ]:
# Setup
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Load env variables
%load_ext dotenv
%dotenv

azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_model_deployment = os.getenv("AZURE_MODEL_DEPLOYMENT")
azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
#api_version = os.getenv("OPENAI_API_VERSION")
model_router = "model-router"

# To use API key, set this to False.  
use_entra_id = True

In [ ]:
os.system("az login")

In [ ]:
# Create clients and helper methods
responses_client = None
chat_client = None

if use_entra_id: 
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(exclude_interactive_browser_credential=False), "https://cognitiveservices.azure.com/.default"
    )
    
    responses_client = AzureOpenAI(
        base_url=azure_oai_endpoint + "openai/v1/",
        api_version="preview",
        azure_ad_token_provider=token_provider
    )

    chat_client = AzureOpenAI(
        azure_endpoint=azure_oai_endpoint,
        api_version="2025-04-01-preview",
        azure_ad_token_provider=token_provider
    )
else:
    responses_client = AzureOpenAI(
        base_url=azure_oai_endpoint + "openai/v1/",
        api_version="preview",
        api_key=azure_oai_key
    )
    
    chat_client = AzureOpenAI(
        azure_endpoint=azure_oai_endpoint,
        api_version="2025-04-01-preview",
        api_key=azure_oai_key
    )


def call_chat_completions_api(prompt, model = azure_model_deployment):    
    response = chat_client.chat.completions.create(
        model = model,
        messages = [
            {"role": "user", "content": prompt},
        ],
    )
    print(response.model_dump_json(indent=2)) 
    return response.choices[0].message.content


def call_model_router(prompt):    
    response = chat_client.chat.completions.create(
        model = model_router,
        messages = [
            {"role": "user", "content": prompt},
        ],
    )
    #print(response.model_dump_json(indent=2))
    
    # Extract the model value
    print("Model used: ", response.model)
    return response.choices[0].message.content

    
def call_model_router_with_image(prompt, image_url):   
    response = chat_client.chat.completions.create(
        model = model_router,
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful assistant." 
            },
            {
                "role": "user", 
                "content": [
    	            {
    	                "type": "text",
    	                "text": prompt
    	            },
    	            {
    	                "type": "image_url",
    	                "image_url": {
                            "url": image_url
                        }
                    } 
               ] 
            },
        ],
    )
    #print(response.model_dump_json(indent=2))
    
    # Extract the model value
    print("Model used: ", response.model)
    return response.choices[0].message.content


def call_responses_api(prompt):
    response = responses_client.responses.create(
        model = azure_model_deployment,
        input = prompt 
    )
    #print(response.model_dump_json(indent=2)) 
    return response.output[1].content[0].text
    

def call_responses_api_with_image_url(prompt, image_url):
    response = responses_client.responses.create(
        model=azure_model_deployment,
        input=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": prompt,
                    },
                    {
                        "type": "input_image",
                        "image_url": image_url
                    }
                ]
            }
        ]
    )
    #print(response.output_text)
    return response.output_text


In [ ]:
# You can use both the responses API and the chat completions API with GPT-5.  
answer = call_responses_api("How many b's are there in blueberry?")
print(answer)

In [ ]:
call_chat_completions_api("How many r are there in strawberry?")

In [ ]:
# Some new parameters!  
response = responses_client.responses.create(
    input="Tell me about the curious case of neural text degeneration",
    model=azure_model_deployment,
    reasoning={
        "effort": "medium",
        "summary": "auto" # auto, concise, or detailed 
    },
    text={
        "verbosity": "low" # New with GPT-5 models
    }
)

print(response.model_dump_json(indent=2))

In [ ]:
# New verbosity parameter

from rich.console import Console
from rich.text import Text
from rich.markdown import Markdown

def pretty_print(verbosity, tokens, output_text):
    console = Console()
    text = Text()
    
    # Print the verbosity in color
    text.append(verbosity, style="bold magenta")
    text.append(f" ({str(tokens)} tokens): ", style="magenta")
    console.print(text)
    md = Markdown(output_text)
    console.print(md)
    console.print("\n")


prompt = "Write an emotional poem about a daughter leaving home for college."

for verbosity in ["low", "medium", "high"]:
    response = responses_client.responses.create(
        model="gpt-5-mini",
        input=prompt,
        text={"verbosity": verbosity}
    )

    # Extract text
    output_text = ""
    for item in response.output:
        if hasattr(item, "content") and item.content != None:
            for content in item.content:
                if hasattr(content, "text"):
                    output_text += content.text

    usage = response.usage
    pretty_print(verbosity, usage.output_tokens, output_text)


In [ ]:
# Addition of "minimal" reasoning effort

prompt = "Explain why the sky is blue, and how this explanation would change if Earth had a thicker atmosphere composed primarily of methane."

for reasoning_effort in ["minimal", "low", "medium", "high"]:
    response = responses_client.responses.create(
        model="gpt-5-mini",
        input=prompt,
        reasoning = {
            "effort": reasoning_effort
        }
    )

    # Extract text
    output_text = ""
    for item in response.output:
        if hasattr(item, "content") and item.content != None:
            for content in item.content:
                if hasattr(content, "text"):
                    output_text += content.text

    usage = response.usage
    pretty_print(reasoning_effort, usage.output_tokens, output_text)

In [ ]:
# Support for custom tools
response = responses_client.responses.create(  
    model="gpt-5",  # replace with your model deployment name  
    tools=[  
        {  
            "type": "custom",
            "name": "lark_tool",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": "start: QUESTION NEWLINE ANSWER\nQUESTION: /[^\\n?]{1,200}\\?/\nNEWLINE: /\\n/\nANSWER: /[^\\n!]{1,200}!/"
            }
        }  
    ],  
    input=[{"role": "user", "content": "Please calculate the area of a circle with radius equal to the number of 'r's in strawberry"}],  
)  

print(response.model_dump_json(indent=2))

In [ ]:
# Building games with good web frontend
def write_to_file(filename, text):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)

checkers_prompt = '''
Create a playable game of checkers in a single page of html.  
+ The checkers should be blue and pink.  Pink always goes first.  
+ The human player should control the pink checkers.  You should write some simple logic for the blue player to always make a valid move.  
+ Blue's turn should be taken immediately after pink moves the checker piece.  
+ Give simple instructions to the user on how to move the pieces in the UI.  
+ The UI should be clean and simple.  
'''
response = call_responses_api(checkers_prompt)
print(response)

num = "1"
filename = "checkers" + num + ".html"
write_to_file(filename, response)

In [ ]:
prompt = '''
Create a playable game of Sudoku in a single page of html.  
+ The difficulty level should be "easy". 
+ Include a way to write notes of possible options for the square, as well as a way to lock in the number that belongs there.  
+ Include a "reset" button to start the game over and change the puzzle.  
+ The UI should be clean and simple.  
'''
response = call_responses_api(prompt)
print(response)

filename = "sudoku" + num + ".html"
write_to_file(filename, response)

In [ ]:
# Building quiz with good web frontend
prompt = '''
Create an online quiz in a single page of html.  
+ The quiz should sort you into one of the 4 Hogwarts houses, based on your answers and the qualities of the students in each house.  
+ The end user should be able to select answers from multiple choice answers.    
+ There should be 10 questions.  After all 10 questions are answered, the webpage should tell you which house you are in with an appropriate graphic to celebrate.
+ The UI should be clean and simple.  Include a scroll bar if the quiz is long enough to need it.  
'''
response = call_responses_api(prompt)
print(response)

filename = "quiz" + num + ".html"
write_to_file(filename, response)

In [ ]:
# Using tools to access data that isn't in training set
# NOTE: this is using OpenAI and not Azure OpenAI.  AOAI doesn't have support for web search yet; reference: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-secure

from openai import OpenAI
openai_client = OpenAI()

prompt = "What is the best song from K Pop Demon Hunters?  How does each song from the movie relate to the Hero's Journey?"

response = openai_client.responses.create(
    model=azure_model_deployment,
    tools=[{"type": "web_search_preview"}],
    input=prompt
)

print(response.output_text)


In [ ]:
# Constraint satisfaction problem - dog medication scheduling
prompt = '''
Our dog Titan has to take 3 medications. He needs 10mg metoclopramide twice per day. He needs 1000mg carafate three times per day, 
but it can't be taken within 2 hours of metoclopramide. Finally, he needs 10mg omeprazole twice per day, and this can be taken with 
the metoclopramide. I need to sleep from midnight-7am. What is the best time to give the dog each medication?
'''
response = call_responses_api(prompt)
print(response)

In [ ]:
# Image understanding with GPT-5 - OCR, spatial reasoning, color identification
wordle_image_url = "https://raw.githubusercontent.com/jennifermarsman/WordleGPT/main/images/Example9Step2.png"
wordle_prompt = "What is a good next word to guess in this Wordle puzzle?"
result = call_responses_api_with_image_url(wordle_prompt, wordle_image_url)
print(result)

In [ ]:
# Image understanding with GPT-5 - counting
image_url = "https://www.royalcourt.no/aim/kongehuset2/files/3/b/3/a61b9ba453c594ec72f0d7148795d5d6a9e5293f0e/3b3a61b9ba453c594ec72f0d7148795d5d6a9e5293f0e.jpg"
prompt = "70 gifts are to be distributed equally among the people in the photo, and the remainder should be given to charity.  How many gifts will each person get?  How much is given to charity?"
result = call_responses_api_with_image_url(prompt, image_url)
print(result)

In [ ]:
# Image understanding with GPT-5 - blind scenario
image_url = "https://ey2msiqxj7z.exactdn.com/wp-content/uploads/2020/04/13162308/0120_Westerly_Reveal_6C_Kitchen_Alt_Angles_Lights_on_15-1-1024x683.jpg?strip=all&lossy=1&ssl=1"
prompt = "I am blind and this is the view in front of me.  I want to prepare a snack of an apple with peanut butter and a glass of milk.  We keep the peanut butter in the pantry.  Give detailed instructions of how to navigate to do this."
result = call_responses_api_with_image_url(prompt, image_url)
print(result)

In [ ]:
# Model selector
prompts = [
    "Hello",
    checkers_prompt
]

for each prompt in prompts:
    reply = call_model_router(prompt)
    print(reply)
    print("\n")

In [ ]:
# Model selector with image url

# Simple example
reply = call_model_router_with_image("Describe this image.", "https://media.licdn.com/dms/image/C4E12AQFuU9fEDEf6UQ/article-cover_image-shrink_600_2000/0/1520216049699?e=2147483647&v=beta&t=cVjZ9psohJpUcNKpgLX-ercyjYHOBHHW-dhVCsjuaHs")
print(reply)
print("\n")

# Complex example
reply = call_model_router_with_image(wordle_prompt, wordle_image_url)
print(reply)